<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

## `pandas` Long Format, Wide Format, Pivot Tables, and Melting
_Instructor: Aymeric Flaisler_
___
<br>

This lesson is all about **transforming data** using `pandas`. Data transformation is the reorganization of your data set's rows and columns into a different, potentially **more useful shape and format**. 

The benefits of transforming your data include **better access to relevant information** and **streamlined data manipulation**. As you become more familiar with data sets and their associated operations, you will develop an intuition and appreciation for when it's better to **work row-wise or column-wise**.

Different data formats are better for different tasks. It takes time and experience to learn the distinctions. But, for now, we'll introduce the **common structures, transformations, and how to apply these transformations**.

### Learning Objectives
- Understand the differences between **long and wide format data**.
- Understand **pivot tables**.
- Practice transforming data between **long and wide** formats.
- Practice creating pivot tables.
- Learn how to avoid **common pitfalls and obstacles** in data transformation with `pandas`.


In [ ]:
import numpy as np
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.max_columns', None)

sns.set_style('darkgrid')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

<a id='load_nerdy'></a>

### Load and Examine the "Nerdy Personality Attributes" Data Set

---

This is a pre-cleaned and modified version of the full "Nerdy Personality Attributes" survey, which asked subjects to rate themselves based on questions related to "nerdiness" as well as more general personality traits such as openness and extraversion. Researches also collected demographic information from the subjects.

You can find the raw data [here](http://personality-testing.info/_rawdata/), along with many other sociological surveys.

In this modified version, for the sake of our example, some of the subjects provided data for the survey but not the demographic variables. Because there are missing values and the data is "messy," we have a data cleaning problem.

**Load the data (which is in wide format).** 

In [ ]:
nerdy_wide_f = 'NPAS_parsed_trunc_wide_missing.csv'

# load data and print the dimensions
nerdy_wide = pd.read_csv(nerdy_wide_f)

This data set is in a familiar format in which each column is a variable and each row contains an observation for that variable, corresponding to a distinct subject.

*Wide format implies that all of the information for one distinct subject **will be represented in the columns corresponding to that row**. A single subject should not be represented in multiple rows of data.*

In [ ]:
# First let's print the header:
nerdy_wide.head()

In [ ]:
nerdy_wide.shape

**Check to see how many null values there are per column.**

*Tips:* An easy way is to use the `.isnull()` method associated with the `.sum()`

In [ ]:
# Now let's count the null values by column:
nerdy_wide.isnull().sum()

**Replace the missing `major` column values with `unknown`.**

In [ ]:
# first create a mask for the missing values in the major column:
#null_mask = nerdy_wide.major.isnull()
# set missing values in major to "unknown":
# nerdy_wide.loc[mask, 'major'] = 'unknown'
nerdy_wide.major.fillna('unknown',inplace=True)

In [ ]:
nerdy_wide.head()

In [ ]:
# if all goes right you should not have any missing values left
print (nerdy_wide.major.isnull().sum())

<a id='long_format'></a>

### Long Format Data



**Load the long format of the same data below.**

In [ ]:
nerdy_long_f = 'NPAS_parsed_trunc_long_missing.csv'

# load long data and print the dimensions
nerdy_long = pd.read_csv(nerdy_long_f)
nerdy_long

You can see that the long format data has far more rows than the wide data set but only three columns.

Below you can view the three columns: `subject_id`, `variable`, and `value`.

**`subject_id:`**
- This is the primary "key" or `ID` column. Each `subject_id` will have corresponding entries in the `variable` column — one for each row.

**`variable:`**
- This column indicates the variable with which the item in the `value` column corresponds.

**`value:`**

- This contains all values for all variables for all IDs. Essentially, every cell in the wide data set except the `subject_id` is listed in this column.

In [ ]:
# print the header:
nerdy_long.head(20)

**Print out the unique values in the `variable` column.**

You can see that the unique values in the `variable` column correspond to the column headers in the wide format data.

*Tips: use the .unique() method*

In [ ]:
# print the unique values in the variable column:
nerdy_long.variable.unique()

In [ ]:
# count the unique subject ids:
len(nerdy_long.subject_id.unique())


**Replace the missing values in `major` with `unknown` in the long format data set.**

The process for replacing data will be different because of the format. Using logical selection masks with `pandas`' `.loc` syntax is the preferable way to do this.

In [ ]:
# Identify the missing values in major:
sum(nerdy_long.value.isnull())

In [ ]:
# replace the missing values for major in the long dataset with "unknown":
major_mask = (nerdy_long.variable == 'major') & (nerdy_long.value.isnull())
nerdy_long.loc[major_mask, 'value'] = 'unknown'

In [ ]:
# check that there is no missing values left:
print(nerdy_long[nerdy_long.variable == 'major'].isnull().sum())

# you should get only 0s

<a id='pivot_tables'></a>

### `Pandas`' `.pivot_table()` Function: Long to Wide Format

---

The `pd.pivot_table()` function is a powerful tool for both transforming data from long to wide format as well as summarizing data with user-supplied functions.

First, we'll look at transforming the long format data back into the wide format using the `.pivot_table()` function.

**Important parameters for the `.pivot_table()` function include:**

> The `pivot_table()` function takes a DataFrame to pivot as its first argument. 
    
- **`columns`**: This is the list of columns in the long format data to be transformed back into columns in the wide format. After pivoting, each unique value in the long format column becomes a header in the wide format.
- **`values`**: A single column indicating the values to use when pivoting and filling the new wide format columns.
- **`index`**: Columns in the long format data that are index variables. These will be left as single columns, not spread out by unique value like in the `columns` parameter.
- **`aggfunc`**: Often `.pivot_table()` is used to perform a summary of the data. `aggfunc` stands for "aggregation function." It's required and defaults to `np.mean()`. You can also insert your own function, which we'll demonstrate below.
- **`fill_value`**: If a cell is missing for the wide format data, this value will fill it in.
    
Next we'll put in our own function — `select_item_or_nan()` — to the `aggfunc` keyword argument. Because my `subject_id` column has a single variable value for each ID, I just want the single element in the long format value cell. My data is messy, so I have to write a function to check for places it could break. 

**Note:** Passed into my function, `x` will be a Series object. I pull out the first element of that using the `.iloc` indexer.

### Let's make sure value has only values:

*Note: The lambda operator or lambda function is used for creating small, one-time and anonymous function objects in Python. This is not the object of this lesson. We will cover it at a later stage. Do not worry about understanding it for now.*

In [ ]:
nerdy_long.shape


In [ ]:
def is_float(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

In [ ]:
# mask with true or false if we can convert to a numerical value
mask = nerdy_long.value.map(lambda x: is_float(x))

#### Now remove non numeric values using the mask

In [ ]:
nerdy_long_only_num = nerdy_long[mask].copy()
#nerdy_long_only_num.reset_index(drop=True, inplace=True)

In [ ]:
nerdy_long_only_num

In [ ]:
nerdy_long_only_num.dtypes

#### Convert the column `value`  from the dataframe `nerdy_long_only_num` to float

In [ ]:
# A:
# nerdy_long_only_num['value'] = ...
nerdy_long_only_num['value'] = nerdy_long_only_num.value.astype(float)

In [ ]:
nerdy_long_only_num.dtypes

#### Finally pivot the data on subject_id and variable using .pivot()

In [ ]:
nerdy_long_only_num.pivot(index='subject_id', columns='variable')

<a id='multiindex'></a>

### MultiIndex/Hierarchical Indices in `pandas`

---

First, let's reload a fresh copy of the data:

In [ ]:
# let's reload the data
nerdy_long = pd.read_csv('nerdy_long.csv')
nerdy_long.pivot(index='subject_id', columns='variable', values='value').head()

In [ ]:
nerdy_long.info()

In [ ]:
nerds_mind = nerdy_long.set_index(['subject_id', 'variable'])
nerds_mind.info()


In [ ]:
nerds_mind

In [ ]:
nerds_mind.unstack().info()

In [ ]:
nerds_mind

In [ ]:
nerds_mind.unstack().stack().reset_index()
